In [1]:
import numpy as np
import torch

import neural_network
from neural_network import sigmoid, sigmoid_der, ReLU, ReLU_der, LeakyReLU, LeakyReLU_der, identity, identity_der, mse, mse_der
import data_sets
import optimizers

In [2]:
# Adam, Runge

np.random.seed(1)
torch.manual_seed(1)

MSE_custom = []
MSE_torch = []

for _ in range(5):
    x_train, x_test, y_train, y_test = data_sets.load_runge_data(100)

    custom_net = neural_network.NeuralNetwork(1, [50, 50, 1], [LeakyReLU, sigmoid, identity], [LeakyReLU_der, sigmoid_der, identity_der], mse, mse_der)
    custom_net.train_network(x_train, y_train, 1, optimizers.Adam(0.01), 1_000)
    print(custom_net.cost(x_test, y_test))
    MSE_custom.append(custom_net.cost(x_test, y_test))

    torch_net = torch.nn.Sequential(
        torch.nn.Linear(1, 50),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(50, 50),
        torch.nn.Sigmoid(),
        torch.nn.Linear(50, 1)
    )
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(torch_net.parameters(), lr=0.01)
    for _ in range(1_000):
        inputs = torch.from_numpy(x_train.astype(np.float32))
        labels = torch.from_numpy(y_train.astype(np.float32))
        optimizer.zero_grad()
        outputs = torch_net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    torch_net.eval()
    with torch.no_grad():
        print(criterion(torch_net(torch.from_numpy(x_test.astype(np.float32))), torch.from_numpy(y_test.astype(np.float32))).item())
        MSE_torch.append(criterion(torch_net(torch.from_numpy(x_test.astype(np.float32))), torch.from_numpy(y_test.astype(np.float32))).item())
    print("-----")

for custom in MSE_custom:
    print(f"${custom:.4f}$", end=" & ")
print()
for torch_mse in MSE_torch:
    print(f"${torch_mse:.4f}$", end=" & ")

0.011290616565537563
0.010784314945340157
-----
0.006962843420107319
0.006406478118151426
-----
0.010204911388337764
0.00912395678460598
-----
0.007168155677114085
0.007427449803799391
-----
0.01134605192321906
0.011339052580296993
-----
$0.0113$ & $0.0070$ & $0.0102$ & $0.0072$ & $0.0113$ & 
$0.0108$ & $0.0064$ & $0.0091$ & $0.0074$ & $0.0113$ & 

In [3]:
# RMSprop, Runge

np.random.seed(5243)
torch.manual_seed(425)

MSE_custom = []
MSE_torch = []

for _ in range(5):
    x_train, x_test, y_train, y_test = data_sets.load_runge_data(100)

    custom_net = neural_network.NeuralNetwork(1, [50, 50, 1], [ReLU, sigmoid, identity], [ReLU_der, sigmoid_der, identity_der], mse, mse_der)
    custom_net.train_network(x_train, y_train, 1, optimizers.RMS_prop(0.0005), 1_000)
    print(custom_net.cost(x_test, y_test))
    MSE_custom.append(custom_net.cost(x_test, y_test))

    torch_net = torch.nn.Sequential(
        torch.nn.Linear(1, 50),
        torch.nn.ReLU(),
        torch.nn.Linear(50, 50),
        torch.nn.Sigmoid(),
        torch.nn.Linear(50, 1)
    )
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.RMSprop(torch_net.parameters(), lr=0.0005, alpha=0.9)
    for _ in range(1_000):
        inputs = torch.from_numpy(x_train.astype(np.float32))
        labels = torch.from_numpy(y_train.astype(np.float32))
        optimizer.zero_grad()
        outputs = torch_net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    torch_net.eval()
    with torch.no_grad():
        print(criterion(torch_net(torch.from_numpy(x_test.astype(np.float32))), torch.from_numpy(y_test.astype(np.float32))).item())
        MSE_torch.append(criterion(torch_net(torch.from_numpy(x_test.astype(np.float32))), torch.from_numpy(y_test.astype(np.float32))).item())
    print("-----")

for custom in MSE_custom:
    print(f"${custom:.4f}$", end=" & ")
print()
for torch_mse in MSE_torch:
    print(f"${torch_mse:.4f}$", end=" & ")

0.009110728639599004
0.0093979611992836
-----
0.009671294181236568
0.010088559240102768
-----
0.00965695459091457
0.011975628323853016
-----
0.015903443324095996
0.014931724406778812
-----
0.012476260804244885
0.015523722395300865
-----
$0.0091$ & $0.0097$ & $0.0097$ & $0.0159$ & $0.0125$ & 
$0.0094$ & $0.0101$ & $0.0120$ & $0.0149$ & $0.0155$ & 

In [ ]:
# Constant, Gaussian 2D

np.random.seed(54366566)
torch.manual_seed(5436576786)

MSE_custom = []
MSE_torch = []

for _ in range(5):
    x_train, x_test, y_train, y_test = data_sets.load_gaussian2D_data(100)

    custom_net = neural_network.NeuralNetwork(2, [50, 50, 1], [sigmoid, sigmoid, identity], [sigmoid_der, sigmoid_der, identity_der], mse, mse_der)
    custom_net.train_network(x_train, y_train, 1, optimizers.Constant(0.001), 1_000)
    print(custom_net.cost(x_test, y_test))
    MSE_custom.append(custom_net.cost(x_test, y_test))

    torch_net = torch.nn.Sequential(
        torch.nn.Linear(2, 50),
        torch.nn.Sigmoid(),
        torch.nn.Linear(50, 50),
        torch.nn.Sigmoid(),
        torch.nn.Linear(50, 1)
    )
    criterion = torch.nn.MSELoss()
    # torch.optim has no 'Constant' optimizer. Use SGD with a fixed learning rate instead.
    optimizer = torch.optim.SGD(torch_net.parameters(), lr=0.001)
    for _ in range(1_000):
        inputs = torch.from_numpy(x_train.astype(np.float32))
        labels = torch.from_numpy(y_train.astype(np.float32))
        optimizer.zero_grad()
        outputs = torch_net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    torch_net.eval()
    with torch.no_grad():
        print(criterion(torch_net(torch.from_numpy(x_test.astype(np.float32))), torch.from_numpy(y_test.astype(np.float32))).item())
        MSE_torch.append(criterion(torch_net(torch.from_numpy(x_test.astype(np.float32))), torch.from_numpy(y_test.astype(np.float32))).item())
        print("-----")

for custom in MSE_custom:
    print(f"${custom:.4f}$", end=" & ")
print()
for torch_mse in MSE_torch:
    print(f"${torch_mse:.4f}$", end=" & ")


0.18296356587438978
0.05309031158685684
-----
0.09145167487556449
0.11899290233850479
-----
0.05589725686615878
0.06944019347429276
-----
0.12846450367829215
0.08233512938022614
-----
0.07259178027131996
0.04444802924990654
-----
$0.1830$ & $0.0915$ & $0.0559$ & $0.1285$ & $0.0726$ & 
$0.0531$ & $0.1190$ & $0.0694$ & $0.0823$ & $0.0444$ & 